In [3]:
import pandas as pd
from pymongo import MongoClient
import praw
from datetime import datetime

In [38]:
# Reddit API credentials
client_id = '_Xy642Bu0_eMzeWappwISQ'
client_secret = 'cbBiXKPfv1jDtkwFbHr5exw83rOCSA'
user_agent = 'ZestycloseNotice3131'

# Create a Reddit instance
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

# MongoDB connection
client = MongoClient('localhost', 27017)
db = client['project-test']
collection = db['climate_posts']

In [39]:
# Subreddit names
conservative_subreddit = reddit.subreddit('Conservative')
liberal_subreddit = reddit.subreddit('Liberal')

# Keywords related to climate change
# Keywords related to climate change
climate_keywords = [
    'climate change', 'global warming', 'environment', 'carbon emissions',
    'greenhouse gases', 'climate crisis', 'climate action', 'climate policy',
    'sustainability', 'renewable energy', 'carbon footprint', 'climate science'
]

conservative_results = conservative_subreddit.search(" OR ".join(climate_keywords), 
                                                     sort='relevance', time_filter='all', syntax='cloudsearch')
liberal_results = liberal_subreddit.search(" OR ".join(climate_keywords), 
                                                     sort='relevance', time_filter='all', syntax='cloudsearch')

In [10]:
# Function to check if a post is about climate change
def is_climate_related(post, keywords):
    title_contains_keyword = any(keyword in post.title.lower() for keyword in keywords)
    selftext_contains_keyword = any(keyword in post.selftext.lower() for keyword in keywords)

    # Get top 20 non-AutoModerator comments
    top_comments = [comment for comment in post.comments if hasattr(comment, 'author') and comment.author and comment.author.name.lower() != 'automoderator'][:20]    
    comments_contain_keyword = any(
        (comment.body is not None) and any(keyword in comment.body.lower() for keyword in keywords)
        for comment in top_comments
    )
    
    print(f"Title: {post.title}, Title Contains Keyword: {title_contains_keyword}")
    print(f"Self-Text: {post.selftext}, Self-Text Contains Keyword: {selftext_contains_keyword}")
    print(f"Comments Contain Keyword: {comments_contain_keyword}")
    
    return title_contains_keyword or selftext_contains_keyword or comments_contain_keyword

# Function to get top twenty posts from a subreddit related to climate change
def get_top_climate_posts(subreddit, keywords):
    top_climate_posts = []
    for submission in subreddit.top():
        if is_climate_related(submission, keywords):
            author_name = submission.author.name if submission.author else 'Unknown'
            post_data = {
                'Subreddit': subreddit.display_name,
                'Title': submission.title,
                'Content': submission.selftext,
                'Author': author_name,
                'Timestamp': datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            }
            top_climate_posts.append(post_data)
    return top_climate_posts

In [40]:
def get_relevant_posts(search_results, subreddit_name):
    posts_data = []

    for submission in search_results:
        post_data = {
            #'Subreddit': subreddit_name,
            'Title': submission.title,
            'Content': submission.selftext,
            'Author': submission.author.name if submission.author else 'Unknown',
            'Timestamp': pd.to_datetime(submission.created_utc, unit='s').strftime('%Y-%m-%d %H:%M:%S')
        }
        posts_data.append(post_data)
    return posts_data

In [41]:
# Get top twenty climate-related posts from r/conservative
conservative_posts = get_relevant_posts(conservative_results, conservative_subreddit)

# Get top twenty climate-related posts from r/liberal
liberal_posts = get_relevant_posts(liberal_results, liberal_subreddit)

# Combine posts from both subreddits
all_posts = conservative_posts + liberal_posts

In [42]:
conservative_posts

[{'Title': "Schumer mocks Texas over deadly energy crisis, blames 'ignored climate change': 'Hope they learned a lesson'",
  'Content': '',
  'Author': 'JesusCumelette',
  'Timestamp': '2021-02-22 00:57:40'},
 {'Title': 'Why did they erase the Medieval Warm Period? Would it prove there is no climate crisis if we knew it was warmer 1k years ago with zero CO2 emissions, and warming is cyclical?',
  'Content': '',
  'Author': 'TallTree9127',
  'Timestamp': '2023-07-25 13:19:26'},
 {'Title': 'Climate Change: A Timeline',
  'Content': '',
  'Author': 'Clatsop',
  'Timestamp': '2019-10-16 11:28:20'},
 {'Title': 'Karine Jean-Pierre: Republicans Still Deny Climate Change Despite "Extreme Heat" Outside',
  'Content': '',
  'Author': 'Vibranium2222',
  'Timestamp': '2023-07-08 14:30:24'},
 {'Title': 'Poll: Voters More Concerned About Gas Prices than Climate Change',
  'Content': '',
  'Author': 'nimobo',
  'Timestamp': '2022-05-10 11:11:44'},
 {'Title': 'Is Climate Change a False Religion?',
  '

In [44]:
len(all_posts)

159

In [45]:
# Insert the data into MongoDB collection
collection.insert_many(all_posts)

# Close MongoDB connection
#client.close()


In [46]:
collection.count_documents({})

159